In [92]:
from ai_core.llm.base import create_chat_model
from langchain_core.messages import HumanMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool

In [93]:
model = create_chat_model(
    llm_api_key="sk-gapk-F14i5UwldXoRfXe6AGxpUSD9G-B0JOUR",
    llm_model="anthropic/claude-3-5-sonnet-20240620",
    # llm_model="openai/gpt-4o-2024-05-13"
)

In [94]:
res = model.invoke("what is the value of 2 * 2?")

In [95]:
res.pretty_print()

================================== Ai Message ==================================

The value of 2 * 2 is 4.

This is a basic multiplication problem:

2 * 2 = 4

In this case, you're multiplying 2 by itself, which gives you 4 as the result. This is one of the fundamental multiplication facts that most people learn early in their math education.


In [19]:
res

AIMessage(content='The value of \\( 2 \\times 2 \\) is 4.', response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 18, 'total_tokens': 34}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_298125635f', 'finish_reason': 'stop', 'logprobs': None}, id='run-7579598d-8dc8-459f-ae6d-655049dcd23e-0', usage_metadata={'input_tokens': 18, 'output_tokens': 16, 'total_tokens': 34})

In [22]:
# streaming
for m in model.stream("what is the value of 2 * 2?"):
    print(m.content, end="")

The value of \(2 \times 2\) is \(4\).

In [ ]:
model.invoke("뭐라고?").pretty_print()

In [ ]:
model.invoke("what is the value of 2132323 * 2232323?").pretty_print()

In [ ]:
2132323 * 2232323

In [43]:
chat_history = []

prompt = ChatPromptTemplate.from_messages(
    [
        (MessagesPlaceholder("history")),
        ('human', '{input}')
    ]
)

In [ ]:
prompt.pretty_print()

In [45]:
model_with_prompt = prompt | model

In [46]:
question = "what is the value of 2 * 2?"
ai_msg_1 = model_with_prompt.invoke({"input": question, "history": chat_history})

In [ ]:
ai_msg_1.pretty_print()

In [48]:
chat_history.extend([question, ai_msg_1])

In [ ]:
ai_msg_2 = model_with_prompt.invoke({"input": "뭐라고?", "history": chat_history})
ai_msg_2.pretty_print()

In [50]:
@tool
def multiply(a: str, b: str) -> int:
    """Multiplies two numbers."""
    return int(a) * int(b)

In [77]:
chat_history = []

tools = [multiply]
model_with_tools = prompt | model.bind_tools(tools)

In [78]:
question = "2132323 * 2232323 은 뭐야?"
ai_msg_3 = model_with_tools.invoke({"input": question, "history": chat_history})

In [ ]:
ai_msg_3.pretty_print()

In [ ]:
tool_call = ai_msg_3.tool_calls[0]
tm = ToolMessage(tool_call_id=tool_call['id'], content=multiply.invoke(tool_call['args']))
tm.pretty_print()

In [ ]:
chat_history.extend([HumanMessage(question), ai_msg_3, tm])
for m in chat_history:
    m.pretty_print()

In [ ]:
res = model.bind_tools(tools).invoke(chat_history)
res.pretty_print()